# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [159]:
# Use this cell to write your code for Task 1
# imports 
import pandas as pd
import numpy as np

# load data
house_sales = pd.read_csv('house_sales.csv')

# check for nulls 
house_sales['city'].isna().sum()

# recheck manually for anomalies
house_sales['city'].unique()

# missing values are represented as --, check for the number
missing_city = house_sales['city'].value_counts().loc['--']

print(f'The number of missing values in the city column are {missing_city}.')

The number of missing values in the city column are 73.


# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [160]:
# Use this cell to write your code for Task 2
# load data
house_df = pd.read_csv('house_sales.csv')

# check for missing values
#print(house_df.isna().sum())

# only months listed has missing values, impute with the mean
avg_months_listed = np.round(house_df['months_listed'].mean(), 1)
house_df['months_listed'].fillna(value = avg_months_listed, inplace = True)

# recheck missing values
#print(house_df.isna().sum())

# fix other errors
# in city column missing values are denotes as --
# replace with Unknown, then turn to categorical
house_df['city'] = house_df['city'].replace({'--' : 'Unknown'}).astype('category')

# transform sale_date to datetime
house_df['sale_date'] = pd.to_datetime(house_df['sale_date'])

# the categories in house type are inconsistent, let's fix that
house_df['house_type'] = house_df['house_type'].replace({'Det.' : 'Detached', 'Semi' : 'Semi-detached', 'Terr.' : 'Terraced'}).astype('category').cat.reorder_categories(new_categories = ['Terraced', 'Semi-detached', 'Detached'])

# fix the area column, convert to float
house_df['area'] = house_df['area'].str.replace('sq.m.', '').str.strip().astype('float')

# house id shld be nominal, change to categorical 
house_df['house_id'] = house_df['house_id'].astype('category')

# confirm dtypes
#print(house_df.dtypes)

# assign to new object
clean_data = house_df

# check that it was cleaned
print(clean_data.isna().sum())
print(clean_data.dtypes)
clean_data.head()

house_id         0
city             0
sale_price       0
sale_date        0
months_listed    0
bedrooms         0
house_type       0
area             0
dtype: int64
house_id               category
city                   category
sale_price                int64
sale_date        datetime64[ns]
months_listed           float64
bedrooms                  int64
house_type             category
area                    float64
dtype: object


house_id        city  sale_price  ... bedrooms     house_type   area
0  1217792  Silvertown       55943  ...        2  Semi-detached  107.8
1  1900913  Silvertown      384677  ...        5       Detached  498.8
2  1174927   Riverford      281707  ...        6       Detached  542.5
3  1773666  Silvertown      373251  ...        6       Detached  528.4
4  1258487  Silvertown      328885  ...        5       Detached  477.1

[5 rows x 8 columns]

# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [161]:
# Use this cell to write your code for Task 3
# load data 
house_data = pd.read_csv('house_sales.csv')

# aggregate the data by bedrooms
price_by_rooms = house_data.groupby('bedrooms', as_index = False).agg(avg_price = ('sale_price', 'mean'), var_price = ('sale_price', 'var'))

# round the results
price_by_rooms['avg_price'] = np.round(price_by_rooms['avg_price'], 1)
price_by_rooms['var_price'] = np.round(price_by_rooms['var_price'], 1)

# view the data
price_by_rooms

bedrooms  avg_price     var_price
0         2    67076.4  5.652896e+08
1         3   154665.1  2.378289e+09
2         4   234704.6  1.725211e+09
3         5   301515.9  2.484328e+09
4         6   375741.3  3.924432e+09

# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [162]:
# Use this cell to write your code for Task 4
# load the data
train = pd.read_csv('train.csv')
validation = pd.read_csv('validation.csv')

# imports
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# clean data for training
# dummy encode the categoricals
to_enc = ['city', 'house_type']
train = pd.get_dummies(data = train, columns = to_enc, dtype = int, drop_first = True)
train.head()

# extract the date info from sale_date
train['sale_date'] = pd.to_datetime(train['sale_date'])
train['sale_year'] = train['sale_date'].dt.year
train['sale_month'] = train['sale_date'].dt.month
train['sale_day'] = train['sale_date'].dt.day

# do the same for validation data
validation = pd.get_dummies(data = validation, columns = to_enc, drop_first = True, dtype = int)
validation['sale_date'] = pd.to_datetime(validation['sale_date'])
validation['sale_year'] = validation['sale_date'].dt.year
validation['sale_month'] = validation['sale_date'].dt.month
validation['sale_day'] = validation['sale_date'].dt.day

# get data ready
X_train = train.drop(['house_id', 'sale_date', 'sale_price'], axis = 'columns')
y_train = train['sale_price']
valid_set = validation.drop(['house_id', 'sale_date'], axis = 'columns')

# fit a baseline model
lr = LinearRegression()
lr.fit(X_train, y_train)

# predict
lr_pred = lr.predict(valid_set)

# save results to a df
base_result = pd.DataFrame({'house_id' : validation['house_id'], 'price' : lr_pred})
base_result.head()

house_id          price
0   1331375  121450.319429
1   1630115  304384.498235
2   1645745  384629.788665
3   1336775  124804.725941
4   1888274  271216.164572

# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [163]:
# Use this cell to write your code for Task 5
# instantiate comparison model
dt = DecisionTreeRegressor(random_state = 42)

# fit the model 
dt.fit(X_train, y_train)

# predict
dt_pred = dt.predict(valid_set)

# save result to a df
compare_result = pd.DataFrame({'house_id' : validation['house_id'], 'price' : dt_pred})
compare_result.head()

house_id     price
0   1331375   82462.0
1   1630115  311847.0
2   1645745  410844.0
3   1336775  105120.0
4   1888274  247463.0